In [ ]:
#Import the required python packages. Skyfield needs to be added through the Python Package Manager.

import os
import datetime
import requests
import json
import random
import arcpy
from skyfield.api import Topos, load, EarthSatellite
ts = load.timescale()

In [15]:
#Set the variables.  Change to match your requirements. This section and the list of bases should be the only two areas you need to edit.
#Start Date and End Date define the date range that you want use in your analysis.
#The grazing angle is used to define the skew angle of the satellites. To calculate, subtract the desired angle from 90. The default is 70.0 (Worldview's skew angle)
#The list variables are the NATO IDs of the optical satellites for Russia and China.
#The file path is the output location that will contain your script generated TLE data, CSVs, and geodatabase.

start_date = ts.utc(2021, 1, 5)
end_date = ts.utc(2021, 1, 6)
grazing_angle = 70.0
russia_list = ["40420", "41394", "44797", "44835", "41032", "42719", "44552", "45608", "37344", "41105", "44903", "43243", "38707", "42825", "43180", "43181", "43876", "43877", "39194", "36095", "43032", "43657", "40358", "44387", "40069", "39177", "40699", "40070", "39186", "40360", "41386", "31792"]
china_list = ["44879", "28890", "44312", "44314", "41899", "33434", "37930", "40137", "38049", "40367", "43491", "37214", "39260", "43010", "41882", "39150", "44622", "43585", "44819", "40118", "41727", "41194", "44703", "40701", "40894", "45625", "45794", "45856", "43259", "43260", "43262", "43461", "43484", "43609", "45721", "37781", "43655", "43011", "44838", "44839", "45624", "45796", "33320", "33321", "38997", "44310", "40959", "40960", "44777", "44836", "45016", "40961", "43946", "41914", "43022", "43023", "43024", "43159", "43160", "43943", "44529", "43942", "43034", "43080", "43146", "43236", "40958", "43485", "44779", "44780", "44781", "44782", "44783", "44486", "37179", "37180", "28220", "33433", "37931", "39455", "41900", "41901", "41907", "41908", "43099", "43100", "41898", "36985", "38256", "40988", "44207", "44209", "44488", "36834", "37165", "37875", "37941", "38257", "38354", "39011", "39012", "39013", "39239", "39240", "39241", "39363", "39410", "40109", "40110", "40111", "40143", "40275", "40305", "40310", "40338", "40339", "40340", "40362", "40878", "41026", "41038", "41473", "42945", "42946", "42947", "45460", "45461", "43028", "43029", "43030", "43081", "43082", "43083", "43170", "43171", "43172", "44449", "44450", "44451", "43275", "43276", "43277", "43642", "33446", "34839", "36110", "36121", "36413", "36414", "36415", "45462", "43643", "41857", "44547", "43909", "43910", "43911", "43912", "43913", "43915", "28470", "43439", "43441", "43442", "43443", "44536", "44537", "44538", "44539", "42761", "42759", "43440", "44534", "38038", "44528", "38046", "41556", "45939"]
file_path = "C:\\1_projects\\138_fedgis2021\\sats\\visible_passes\\"


#These variables should be left as is.

russia_sat_count = 0
china_sat_count = 0

In [16]:
#TLE data contains all the information needed for analyzing oribital paths.
#It's generated every day and the latest version should be used to ensure the most accurate predictions.
#This code copies a daily TLE release as json to your working folder.

original_tle_file = requests.get("http://stuffin.space/TLE.json", stream=True)
copy_tle_file = open(file_path + "TLE.json", "w")
copy_tle_file.write(original_tle_file.text)
copy_tle_file.close()

In [17]:
#This code creates a custom TLE file that only contains the Russian and Chinese optical satellites.

local_tle_file = open(file_path + "TLE.json",'r')
subset_tle_file = open(file_path + "TLE_Subset.tle", "w")
jsonObject = json.loads(local_tle_file.read())
for sat_obj in jsonObject:
    nato = sat_obj["TLE_LINE2"].split(" ")
    if nato[1] in russia_list:
        subset_tle_file.write(sat_obj["OBJECT_NAME"] + ":::" + nato[1] + ":::" + sat_obj["TLE_LINE1"] + ":::" + sat_obj["TLE_LINE2"] + ":::" + "Russia\n")
    if nato[1] in china_list:
        subset_tle_file.write(sat_obj["OBJECT_NAME"] + ":::" + nato[1] + ":::" + sat_obj["TLE_LINE1"] + ":::" + sat_obj["TLE_LINE2"] + ":::" + "China\n")
local_tle_file.close()
subset_tle_file.close()

In [18]:
#This function calculates the actual image windows for each pass.

def find_viewing_times_and_locations(ts, t0, t1, satellite, my_location, sat_name, country, nato_id, base_name):
    global russia_sat_count
    global china_sat_count
    visible_passes = open(file_path + base_name + " Visible Passes.csv", "a")
    t, events = satellite.find_events(my_location, t0, t1, altitude_degrees=grazing_angle)
    identifier = random.getrandbits(64)
    for ti, event in zip(t, events):
        geocentric = satellite.at(ti)
        subpoint = geocentric.subpoint()
        ddlatd = str(subpoint.latitude).split("deg ")
        ddlatm = float(str(ddlatd[1]).split("' ")[0])
        ddlats = float(str(ddlatd[1]).split("' ")[1].replace('"',''))
        ddlond = str(subpoint.longitude).split("deg ")
        ddlonm = float(str(ddlond[1]).split("' ")[0])
        ddlons = float(str(ddlond[1]).split("' ")[1].replace('"',''))
        lat = abs(float(ddlatd[0])) + (ddlatm/60) + (ddlats/3600)
        lng = abs(float(ddlond[0])) + (ddlonm/60) + (ddlons/3600)
        visiblity = "appearance"
        if event == 1:
            #identifier = identifier + 1
            if country == "Russia":
                russia_sat_count = russia_sat_count + 1
            if country == "China":
                china_sat_count = china_sat_count + 1
        if event == 2:
            visiblity = "disappearance"
        if str(subpoint.latitude).startswith("-"):
            lat = lat * -1
        if str(subpoint.longitude).startswith("-"):
            lng = lng * -1
        if event != 1:
            if lat != "":
                visible_passes.write(base_name.replace(",", "") + "," + sat_name.replace(",", "") + "," + country + "," + str(nato_id) + "," + str(lat) + "," + str(lng) + "," + "a" + str(identifier) + "," + visiblity + "," + ti.utc_strftime('%m/%d/%Y %H:%M:%S') + "\n")
    visible_passes.close()

In [19]:
#This function creates a CSV for each location that contains entires for when an image window begins and when it ends. 

def calculate_passes_by_location(base_name, coords):
    global russia_sat_count
    global china_sat_count
    tle_file = open(file_path + "TLE_Subset.tle", "r")
    visible_passes = open(file_path + base_name + " Visible Passes.csv", "w")
    visible_passes.write("base_name, sat_name, country, nato_id, lat, long, id, phase, date\n")
    visible_passes.close()
    for line in tle_file.readlines():
        split_data = line.split(":::")
        sat_name = split_data[0].replace(",", "")
        line1 = split_data[2]
        line2 = split_data[3]
        satellite = EarthSatellite(line1, line2, sat_name, ts)
        find_viewing_times_and_locations(ts, start_date, end_date, satellite, coords, sat_name.replace(",", ""), split_data[4].replace("\n", ""), split_data[1], base_name)
    print (str(russia_sat_count) + " Russian satellites will have an imaging window of " + base_name + ".")
    print (str(china_sat_count) + " Chinese satellites will have an imaging window of " + base_name + ".")
    russia_sat_count = 0
    china_sat_count = 0

In [21]:
#This is the other section that has variables that need to be edited.
#Add as many locations as you would like while using the format below.

calculate_passes_by_location("Joint Base Anacostia-Bolling", Topos('38.848002 N', '77.012290 W'))
calculate_passes_by_location("MacDill AFB", Topos('27.8531 N', '82.5030 W'))
calculate_passes_by_location("Peterson AFB", Topos('38.8239 N', '104.7001 W'))
calculate_passes_by_location("Buckley AFB", Topos('39.7240 N', '104.7835 W'))
calculate_passes_by_location("Offut AFB", Topos('41.1243 N', '95.9146 W'))
print ("\nAnalysis Complete.")

10 Russian satellites will have an imaging window of Joint Base Anacostia-Bolling.
75 Chinese satellites will have an imaging window of Joint Base Anacostia-Bolling.
8 Russian satellites will have an imaging window of MacDill AFB.
76 Chinese satellites will have an imaging window of MacDill AFB.
14 Russian satellites will have an imaging window of Peterson AFB.
64 Chinese satellites will have an imaging window of Peterson AFB.
14 Russian satellites will have an imaging window of Buckley AFB.
70 Chinese satellites will have an imaging window of Buckley AFB.
7 Russian satellites will have an imaging window of Offut AFB.
85 Chinese satellites will have an imaging window of Offut AFB.

Analysis Complete.


In [22]:
#This function takes the output from the previous section and turns it into a geodatabase.
#A feature class is created for each base that contains time-enabled lines for each image window.

def symbolize_passes(base_name, gbd_path):
    arcpy.env.workspace = file_path

    SR = arcpy.SpatialReference(4326)
    fc_name = base_name.replace("-", "_")
    fc_name = fc_name.replace(" ", "_")
    new_polyline = arcpy.CreateFeatureclass_management(gdb_path, fc_name, 'POLYLINE', spatial_reference=SR)
    fc = new_polyline[0]
    arcpy.AddField_management(fc, "Base_Name", "TEXT", "", "", 100, "Base_Name")
    arcpy.AddField_management(fc, "Sat_Name", "TEXT", "", "", 100, "Sat_Name")
    arcpy.AddField_management(fc, "Country", "TEXT", "", "", 100, "Country")
    arcpy.AddField_management(fc, "NATO_ID", "TEXT", "", "", 100, "NATO_ID")
    arcpy.AddField_management(fc, "Appear", "DATE", "", "", 100, "Appear")
    arcpy.AddField_management(fc, "Disappear", "DATE", "", "", 100, "Disappear")
    
    csv = open(file_path + file_name, "r")
    line_counter = -1
    appear_x = "a"
    appear_y = "a"
    appear_time = "a"
    disappear_time = "a"
    for line in csv:
        line_counter = line_counter + 1
        if line_counter == 0:
            print (base_name + " is exporting.")
        elif (line_counter % 2) == 0:
            data = line.split(",")
            with arcpy.da.InsertCursor(fc, ['SHAPE@', 'Base_Name', "Sat_Name", "Country", "NATO_ID", "Appear", "Disappear"]) as cursor:
                coordinates = [(data[5],data[4]),(appear_x, appear_y)]
                cursor.insertRow((coordinates, data[0], data[1], data[2], data[3], appear_time, data[8]))
        else:
            data = line.split(",")
            appear_x = data[5]
            appear_y = data[4]
            appear_time = data[8]

In [23]:
#This code loops through all the data in your working folder and turns it into geospatial data.
#When it's completed all of the image windows should be available in your map.

gdb_path = file_path + datetime.datetime.now().strftime("%B_%d_%Y_%I_%M_%S%p") + '.gdb'
if os.path.isfile(gdb_path) == False:
    arcpy.CreateFileGDB_management(file_path, datetime.datetime.now().strftime("%B_%d_%Y_%I_%M_%S%p") + '.gdb')
        
for file_name in os.listdir(file_path):
    if file_name.endswith("Visible Passes.csv"):
        symbolize_passes(file_name.replace(".csv", ""), gdb_path)
        
print ("\nExport complete.")

Buckley AFB Visible Passes is exporting.
Joint Base Anacostia-Bolling Visible Passes is exporting.
MacDill AFB Visible Passes is exporting.
Offut AFB Visible Passes is exporting.
Peterson AFB Visible Passes is exporting.

Export complete.
